# Prepare the Env

In [1]:
!pip install sagemaker==2.174.0 accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 datasets==2.14.2 huggingface-hub==0.16.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.4/857.4 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:0

# Setup your SageMaker environment

In [ ]:
import sagemaker
sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

In [ ]:
sess.default_bucket()

# Preprocess data

Please download the data from Kaggle: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data if the wget line below does not work for you
You can manually download the data from it's homepage or use the Kaggle CLI tool (follow the instructions here: https://www.kaggle.com/docs/api).

Please keep the downloaded zip file as data.zip in the current working directory

In [2]:
!wget -O "data.zip" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/8076/44219/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1691642182&Signature=YpZ4ClnxcMbsKe2FbbOE3deKUzL%2FR%2F7IlvvlNyr4JnKpwBazZbAVvQJIoI6bRSTI%2BXTdL5Nkp4ZOIajdZDp2f9k4lvIAnGJBLIqeAxIY%2FTraQCeF5XwdE5%2B48dEc%2BZQpGJngTM%2FZW4ghsnO%2FUOKpBKyKav%2F9QSGYtotYJORGpWpgKRLT55Q9mmSZON%2FKK%2BAZDRgvjmXecOOGq948CCDpk5x1QCRk8miGMTLd7aF7B%2BaHHNEEjey7A%2Ffd4dySeMgy49nn5uY3yg4XJQJLS27Nrcmk1M5YdC1O2gfZeOS86Tf%2BcjdHvHYU1to0TSNSw6sxaC0BLNpMVfE6uwfK7lk58A%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-toxic-comment-classification-challenge.zip"
import os
import io
import zipfile

def extract(filename, path="."):
  z = zipfile.ZipFile(filename) # open the outer zip file
  for f in z.namelist(): # loop through the files in the outer zip file
    full_path = os.path.join(path, f) # get the full path of the file
    if f.endswith("/"): # check if the file is a directory
      os.makedirs(full_path, exist_ok=True) # create the directory if it doesn't exist
    else:
      if os.path.exists(full_path): # check if the file already exists
        new_name = full_path + ".bak" # create a new name with a .bak extension
        os.rename(full_path, new_name) # rename the existing file
      z.extract(f, path) # extract the file into the given path
      if f.endswith(".zip"): # check if the file is also a zip file
        dir_name = os.path.dirname(full_path) # get the directory name of the outer zip file
        base_name = os.path.basename(full_path) # get the file name of the inner zip file without the extension
        new_path = os.path.join(dir_name, base_name) # create a new path for the inner zip file
        os.rename(full_path, new_path) # rename the inner zip file to remove the .zip extension
        extract(new_path, dir_name) # recursively unzip it into the same directory as the outer zip file
extract("data.zip","data")


--2023-08-07 07:48:44--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/8076/44219/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1691642182&Signature=YpZ4ClnxcMbsKe2FbbOE3deKUzL%2FR%2F7IlvvlNyr4JnKpwBazZbAVvQJIoI6bRSTI%2BXTdL5Nkp4ZOIajdZDp2f9k4lvIAnGJBLIqeAxIY%2FTraQCeF5XwdE5%2B48dEc%2BZQpGJngTM%2FZW4ghsnO%2FUOKpBKyKav%2F9QSGYtotYJORGpWpgKRLT55Q9mmSZON%2FKK%2BAZDRgvjmXecOOGq948CCDpk5x1QCRk8miGMTLd7aF7B%2BaHHNEEjey7A%2Ffd4dySeMgy49nn5uY3yg4XJQJLS27Nrcmk1M5YdC1O2gfZeOS86Tf%2BcjdHvHYU1to0TSNSw6sxaC0BLNpMVfE6uwfK7lk58A%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-toxic-comment-classification-challenge.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.128, 172.217.194.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55201987 (53M) [application/zip]
Saving to:

In [3]:
# path to train.csv test.csv and test_labels.csv
from pathlib import Path
from datasets import load_dataset
data_dir = Path("./data")
ds = (load_dataset("jigsaw_toxicity_pred", data_dir=data_dir, split='train')
        .train_test_split(train_size=800, test_size=200))
# create labels column
cols = ds["train"].column_names
ds = ds.map(lambda x : {"labels": [float(x[c]) for c in cols if c != "comment_text"]})
train_dataset = ds['train']
test_dataset = ds['test']

Generating train split:   0%|          | 0/159571 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/63978 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [4]:
train_dataset

Dataset({
    features: ['comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'labels'],
    num_rows: 800
})

In [5]:
from transformers import AutoTokenizer
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# create tokenization function
def tokenize(batch):
    return tokenizer(batch["comment_text"], padding="max_length", truncation=True)

# tokenize train and test datasets
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set dataset format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [13]:
train_dataset[0]['labels']

tensor([0., 0., 0., 0., 0., 0.])

In [7]:
# save train_dataset to local
s3_prefix = "data/jisaw"
training_input_path = f'./processed_data/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path)

# save test_dataset to s3
test_input_path = f'./processed_data/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path)

Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# save train_dataset to s3
s3_prefix = "data/jisaw"
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path)

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

# Prepare a training job

In [ ]:
"""
The Hugging Face extension for the SageMaker Python SDK means we can benefit from fully-managed EC2 spot instances. This can help you save up to 90% of training costs!

Note: Unless your training job completes quickly, we recommend you use checkpointing with managed spot training. In this case, you need to define the checkpoint_s3_uri.

Set use_spot_instances=True and define your max_wait and max_run time in the Estimator to use spot instances
"""
from sagemaker.huggingface import HuggingFace
# define metrics definitions
metric_definitions = [
    {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
    {"Name": "eval_accuracy", "Regex": "eval_accuracy.*=\D*(.*?)$"},
    {"Name": "eval_loss", "Regex": "eval_loss.*=\D*(.*?)$"},
]
# hyperparameters which are passed to the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 32,
                 'model_name':'distilbert-base-uncased',
                 'output_dir':'/opt/ml/checkpoints',
                 "num_labels": 6 # the number of labels is 6 in our settings when using jigsaw_toxicity_pred dataset
                 }


# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p2.xlarge',
        instance_count=1,
        # use_spot_instances=True,
        # max_wait=3600,
        # max_run=1000,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        metric_definitions=metric_definitions,
        hyperparameters = hyperparameters)

In [ ]:
huggingface_estimator.fit({"train": training_input_path, "test": test_input_path})

# Access trained model

In [ ]:
from sagemaker.s3 import S3Downloader

S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # S3 URI where the trained model is located
    local_path='.',                          # local path where *.targ.gz is saved
    sagemaker_session=sess                   # SageMaker session used for training the model
)

# Distributed training

## Data parallelism

In [ ]:
# configuration for running training on smdistributed data parallel
from sagemaker.huggingface import HuggingFace
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3dn.24xlarge',
        instance_count=2,
        role=role,
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        hyperparameters = hyperparameters,
        distribution = distribution
)
huggingface_estimator.fit({"train": training_input_path, "test": test_input_path})

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## Model parallelism

In [ ]:
# configuration for running training on smdistributed model parallel
from sagemaker.huggingface import HuggingFace
mpi_options = {
    "enabled" : True,
    "processes_per_host" : 8
}

smp_options = {
    "enabled":True,
    "parameters": {
        "microbatches": 4,
        "placement_strategy": "spread",
        "pipeline": "interleaved",
        "optimize": "speed",
        "partitions": 4,
        "ddp": True,
    }
}

distribution={
    "smdistributed": {"modelparallel": smp_options},
    "mpi": mpi_options
}

 # create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3dn.24xlarge',
        instance_count=2,
        checkpoint_s3_uri=f's3://{sess.default_bucket()}/checkpoints',
        role=role,
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        hyperparameters = hyperparameters,
        distribution = distribution
)